#Tree-sitter for Syntax-aware Tokenization

In [1]:
!pip uninstall -y tree_sitter
!pip install tree_sitter==0.20.0

Found existing installation: tree_sitter 0.20.0
Uninstalling tree_sitter-0.20.0:
  Successfully uninstalled tree_sitter-0.20.0
  Using cached tree_sitter-0.20.0-cp312-cp312-linux_x86_64.whl


In [9]:
!git clone https://github.com/JoranHonig/tree-sitter-solidity.git

fatal: destination path 'tree-sitter-solidity' already exists and is not an empty directory.


#Hugging Face Transformers for AutoTokenizer adn CodeBERT

In [4]:
!pip install transformers

#Tokenizing Solidity Code

##Build Solidity Parser:only done once

In [11]:
from tree_sitter import Language

Language.build_library(
  'build/my-languages.so',
  ['tree-sitter-solidity']
)

False

In [13]:
from tree_sitter import Parser, Language

SOLIDITY = Language('build/my-languages.so', 'solidity')
parser = Parser()
parser.set_language(SOLIDITY)

In [ ]:
code_vuln1 = b"""
// SPDX-License-Identifier: MIT
pragma solidity ^0.6.0;

contract Token {
    mapping(address => uint256) balances;
    uint256 public totalSupply;

    constructor(uint256 _initialSupply) public {
        balances[msg.sender] = totalSupply = _initialSupply;
    }

    function transfer(address _to, uint256 _value) public returns (bool) {
        require(balances[msg.sender] - _value >= 0);
        balances[msg.sender] -= _value;
        balances[_to] += _value;
        return true;
    }

    function balanceOf(address _owner) public view returns (uint256 balance) {
        return balances[_owner];
    }
}
"""

In [ ]:
code_sec1 = b"""
// SPDX-License-Identifier: MIT
pragma solidity ^0.6.0;

contract Token {
    mapping(address => uint256) balances;
    uint256 public totalSupply;

    constructor(uint256 _initialSupply) public {
        balances[msg.sender] = totalSupply = _initialSupply;
    }

    function transfer(address _to, uint256 _value) public returns (bool) {
        require(balances[msg.sender] - _value >= 0);
        balances[msg.sender] -= _value;
        balances[_to] += _value;
        return true;
    }

    function balanceOf(address _owner) public view returns (uint256 balance) {
        return balances[_owner];
    }
}
"""

In [14]:

tree = parser.parse(code_vuln1)

def extract_tokens(node, code):
    tokens = []
    if node.child_count == 0:
        token_text = code[node.start_byte:node.end_byte].decode('utf-8')
        tokens.append((node.type, token_text))
    for child in node.children:
        tokens.extend(extract_tokens(child, code))
    return tokens

tokens = extract_tokens(tree.root_node, code_vuln1)
for token_type, token_text in tokens:
    print(f"{token_type}: {token_text}")

comment: // SPDX-License-Identifier: MIT
pragma: pragma
solidity: solidity
^:  ^
solidity_version: 0.6.0
contract: contract
identifier: Token
{: {
mapping: mapping
(: (
address: address
=>: =>
uint256: uint256
): )
identifier: balances
uint256: uint256
public: public
identifier: totalSupply
constructor: constructor
(: (
uint256: uint256
identifier: _initialSupply
): )
public: public
{: {
identifier: balances
[: [
identifier: msg
.: .
identifier: sender
]: ]
=: =
identifier: totalSupply
=: =
identifier: _initialSupply
}: }
function: function
identifier: transfer
(: (
address: address
identifier: _to
,: ,
uint256: uint256
identifier: _value
): )
public: public
returns: returns
(: (
bool: bool
): )
{: {
identifier: require
(: (
identifier: balances
[: [
identifier: msg
.: .
identifier: sender
]: ]
-: -
identifier: _value
>=: >=
number_literal: 0
): )
identifier: balances
[: [
identifier: msg
.: .
identifier: sender
]: ]
-=: -=
identifier: _value
identifier: balances
[: [
identifier: _to
]